# 1. Kết nối tới drive , import một số thư viện cần thiết

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy.polynomial.polynomial as poly
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns


from datetime import datetime
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import Lasso
from sklearn.utils import column_or_1d

from keras.models import Sequential
from keras.layers import Dense, GRU
from keras.regularizers import L1L2

In [ ]:
FPT = pd.read_csv("/content/drive/MyDrive/Data pttk/VCB-doan.csv")

In [ ]:
VCB

In [ ]:
VCB.info()

In [ ]:
# Convert "Date" column to datetime type
VCB['Date'] = pd.to_datetime(VCB['Date'])

# Convert "Vol" and "Change" columns to numeric types
VCB['Vol'] = pd.to_numeric(VCB['Vol'], errors='coerce')
VCB['Change'] = pd.to_numeric(VCB['Change'], errors='coerce')

# Sort the DataFrame by the "Date" column
VCB = VCB.sort_values('Date')


# 2. Tiền xử lý dữ liệu

### Loại bỏ một số dữ liệu trống trong dataset

In [ ]:
# Drop missing values
VCB = VCB.dropna()

In [ ]:
VCB1 = VCB.reset_index()['Close']

In [ ]:
VCB['Index'] = np.arange(len(VCB.index))

<h1> 3.Scale dữ liệu

In [ ]:
scaler=MinMaxScaler(feature_range=(0,1))
VCB1=scaler.fit_transform(np.array(VCB1).reshape(-1,1))

# 4. Chia tập dữ liệu để train-test-validate theo tỉ lệ 7:2:1

In [ ]:
# Split the data into training, testing, and validation sets
train_size = int(0.7 * len(VCB1))
test_size = int(0.2 * len(VCB1))
val_size = len(VCB1) - train_size - test_size

In [ ]:
train_data = VCB1[:train_size]
test_data = VCB1[train_size:train_size+test_size]
val_data = VCB1[train_size+test_size:]

<h1>Hàm Create Dataset

In [ ]:
#Chuyển mảng giá trị của dataset thành ma trận
def create_dataset(dataset, time_step=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, X=0,1,2,3-----99   Y=100
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [ ]:
import numpy
#Reshape into X=t,t+1, t+2... t+99 and Y= t+100
time_step =100
X_train, y_train = create_dataset(train_data, time_step)
X_val, yval = create_dataset(val_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)

In [ ]:
#Reshape input to be [samples, time steps, features] which is required for GRU
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)
X_val = X_val.reshape(X_val.shape[0],X_val.shape[1] , 1)

In [ ]:
# Train a Lasso model
lasso = Lasso(alpha=0.01)
lasso.fit(X_train.reshape(X_train.shape[0], -1), y_train)

# Get Lasso coefficients as additional features
lasso_features_train = lasso.predict(X_train.reshape(X_train.shape[0], -1))
lasso_features_test = lasso.predict(X_test.reshape(X_test.shape[0], -1))
lasso_features_val = lasso.predict(X_val.reshape(X_val.shape[0], -1))

# Reshape Lasso features to be compatible for concatenation
lasso_features_train = lasso_features_train.reshape(-1, 1, 1)
lasso_features_test = lasso_features_test.reshape(-1, 1, 1)
lasso_features_val = lasso_features_val.reshape(-1, 1, 1)

# Concatenate Lasso features with original features
X_train_combined = np.concatenate([X_train, lasso_features_train.repeat(time_step, axis=1)], axis=2)
X_test_combined = np.concatenate([X_test, lasso_features_test.repeat(time_step, axis=1)], axis=2)
X_val_combined = np.concatenate([X_val, lasso_features_val.repeat(time_step, axis=1)], axis=2)


In [ ]:
# Define GRU model with L1L2 regularization (mimicking Lasso)
model = Sequential()
model.add(GRU(64, input_shape=(time_step, 2)))  # 2 features: original + Lasso
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [1]:
#Model fitting
model.fit(X_train_combined, y_train, validation_data=(X_test_combined, y_test), epochs=100, batch_size=64, verbose=1)

NameError: ignored

<h1>Dự báo dữ liệu test, val

In [2]:
train_predict = model.predict(X_train_combined)
y_pred = model.predict(X_test_combined)
y_pred_val = model.predict(X_val_combined)


NameError: ignored

<h1>Chuẩn hóa dữ liệu

In [ ]:
#Chuẩn hóa dữ liệu y_pred, y_pred_val
train_predict=scaler.inverse_transform(train_predict)
y_pred=scaler.inverse_transform(y_pred)
y_pred_val=scaler.inverse_transform(y_pred_val)

In [3]:
ytest = scaler.inverse_transform(ytest.reshape(-1,1))
ytest

NameError: ignored

In [ ]:
yval = scaler.inverse_transform(yval.reshape(-1,1))
yval

<h1>Đánh giá độ chính xác của thuật toán

In [4]:
# RMSE
valid_rmse = np.sqrt(np.mean((y_pred_val - yval)**2))
test_rmse = np.sqrt(np.mean((y_pred - ytest)**2))
print('Validation RMSE:', valid_rmse)
print('Testing RMSE:', test_rmse)

NameError: ignored

In [ ]:
#MAE
valid_mae = mean_absolute_error(yval,y_pred_val)
test_mae = mean_absolute_error(ytest, y_pred)
print('Validation MAE:', valid_mae)
print('Testing MAE:', test_mae)

In [5]:
#MAPE
valid_mape = np.mean(np.abs((yval - y_pred_val) / yval))*100
test_mape = np.mean(np.abs((ytest - y_pred) / ytest))*100
print('Validation MAPE:', valid_mape)
print('Testing MAPE:', test_mape)

NameError: ignored

<h1>Quá trình dự đoán giá cổ phiếu trong 30 ngày tiếp theo

In [ ]:

lst_output = []
n_steps = 30  # Adjust the sequence length as needed
i = 0

while i < 30:
    if len(temp_input) >= n_steps:
        x_input = np.array(temp_input[-n_steps:])
        x_input = x_input.reshape((1, n_steps, 1))

        # Print for debugging
        print("{} day input {}".format(i, x_input))

        yhat = model.predict(x_input, verbose=0)

        # Print for debugging
        print("{} day output {}".format(i, yhat))

        temp_input.extend(yhat[0].tolist())
        temp_input = temp_input[1:]
        lst_output.extend(yhat.tolist())
        i = i + 1
    else:
        x_input = x_input.reshape((1, len(temp_input), 1))

        # Print for debugging
        print("{} day input {}".format(i, x_input))

        yhat = model.predict(x_input, verbose=0)

        # Print for debugging
        print("{} day output {}".format(i, yhat))

        temp_input.extend(yhat[0].tolist())
        i = i + 1

print(lst_output)


In [6]:
print(scaler.inverse_transform(lst_output))

NameError: ignored

<h1>Vẽ hình mô tả dự đoán

In [ ]:
#Drawing plot
train_data_index = pd.RangeIndex(start=0, stop=train_size, step=1)
plt.plot(scaler.inverse_transform(train_data))
test_data_index = pd.RangeIndex(start=train_size, stop=train_size+test_size, step=1)
plt.plot(test_data_index,scaler.inverse_transform(test_data))
test_data_index = pd.RangeIndex(start=train_size+101, stop=train_size+test_size, step=1)
plt.plot(test_data_index,(y_pred))
val_data_index = pd.RangeIndex(start=train_size+test_size, stop=train_size+test_size+val_size, step=1)
plt.plot(val_data_index,scaler.inverse_transform(val_data))
val_data_index = pd.RangeIndex(start=train_size+test_size+101, stop=train_size+test_size+val_size, step=1)
plt.plot(val_data_index,y_pred_val)
prediect_data_index = pd.RangeIndex(start=len(VCB1)-1, stop=len(VCB1)+29, step=1)
plt.plot(prediect_data_index,scaler.inverse_transform(lst_output))
plt.legend(['Train','Test','Test Predictions ','Validate','Validate Predictions','Next30days Predictions'])
plt.title("Dự đoán giá cổ phiếu VCB trong 30 ngày kế tiếp theo tỉ lệ 7-2-1")
plt.show()

# 5. Chia tập dữ liệu để train-test-validate theo tỉ lệ 6:3:1

In [ ]:
# Split the data into training, testing, and validation sets
train_size = int(0.6 * len(VCB1))
test_size = int(0.3 * len(VCB1))
val_size = len(VCB1) - train_size - test_size

In [7]:
train_data = VCB1[:train_size]
test_data = VCB1[train_size:train_size+test_size]
val_data = VCB1[train_size+test_size:]

NameError: ignored

<h1>Hàm Create Dataset

In [8]:
#Chuyển mảng giá trị của dataset thành ma trận
def create_dataset(dataset, time_step=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, X=0,1,2,3-----99   Y=100
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [9]:
import numpy
#Reshape into X=t,t+1, t+2... t+99 and Y= t+100
time_step =100
X_train, y_train = create_dataset(train_data, time_step)
X_val, yval = create_dataset(val_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)

NameError: ignored

In [10]:
#Reshape input to be [samples, time steps, features] which is required for GRU
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)
X_val = X_val.reshape(X_val.shape[0],X_val.shape[1] , 1)

NameError: ignored

In [ ]:
#Define GRU model
model = Sequential()
model.add(GRU(64, input_shape=(time_step, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
#Model fitting
model.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=100,batch_size=64,verbose=1)

<h1>Dự báo dữ liệu test, val

In [ ]:
train_predict=model.predict(X_train)
y_pred=model.predict(X_test)
y_pred_val=model.predict(X_val)

<h1>Chuẩn hóa dữ liệu

In [ ]:
#Chuẩn hóa dữ liệu y_pred, y_pred_val
train_predict=scaler.inverse_transform(train_predict)
y_pred=scaler.inverse_transform(y_pred)
y_pred_val=scaler.inverse_transform(y_pred_val)

In [ ]:
ytest = scaler.inverse_transform(ytest.reshape(-1,1))
ytest

In [ ]:
yval = scaler.inverse_transform(yval.reshape(-1,1))
yval

<h1>Đánh giá độ chính xác của thuật toán

In [ ]:
# RMSE
valid_rmse = np.sqrt(np.mean((y_pred_val - yval)**2))
test_rmse = np.sqrt(np.mean((y_pred - ytest)**2))
print('Validation RMSE:', valid_rmse)
print('Testing RMSE:', test_rmse)

In [ ]:
#MAE
valid_mae = mean_absolute_error(yval,y_pred_val)
test_mae = mean_absolute_error(ytest, y_pred)
print('Validation MAE:', valid_mae)
print('Testing MAE:', test_mae)

In [ ]:
#MAPE
valid_mape = np.mean(np.abs((yval - y_pred_val) / yval))*100
test_mape = np.mean(np.abs((ytest - y_pred) / ytest))*100
print('Validation MAPE:', valid_mape)
print('Testing MAPE:', test_mape)

<h1>Quá trình dự đoán giá cổ phiếu trong 30 ngày tiếp theo

In [ ]:

lst_output = []
n_steps = 30  # Adjust the sequence length as needed
i = 0

while i < 30:
    if len(temp_input) >= n_steps:
        x_input = np.array(temp_input[-n_steps:])
        x_input = x_input.reshape((1, n_steps, 1))

        # Print for debugging
        print("{} day input {}".format(i, x_input))

        yhat = model.predict(x_input, verbose=0)

        # Print for debugging
        print("{} day output {}".format(i, yhat))

        temp_input.extend(yhat[0].tolist())
        temp_input = temp_input[1:]
        lst_output.extend(yhat.tolist())
        i = i + 1
    else:
        x_input = x_input.reshape((1, len(temp_input), 1))

        # Print for debugging
        print("{} day input {}".format(i, x_input))

        yhat = model.predict(x_input, verbose=0)

        # Print for debugging
        print("{} day output {}".format(i, yhat))

        temp_input.extend(yhat[0].tolist())
        i = i + 1

print(lst_output)


In [ ]:
print(scaler.inverse_transform(lst_output))

<h1>Vẽ hình mô tả dự đoán

In [ ]:
#Drawing plot
train_data_index = pd.RangeIndex(start=0, stop=train_size, step=1)
plt.plot(scaler.inverse_transform(train_data))
test_data_index = pd.RangeIndex(start=train_size, stop=train_size+test_size, step=1)
plt.plot(test_data_index,scaler.inverse_transform(test_data))
test_data_index = pd.RangeIndex(start=train_size+101, stop=train_size+test_size, step=1)
plt.plot(test_data_index,(y_pred))
val_data_index = pd.RangeIndex(start=train_size+test_size, stop=train_size+test_size+val_size, step=1)
plt.plot(val_data_index,scaler.inverse_transform(val_data))
val_data_index = pd.RangeIndex(start=train_size+test_size+101, stop=train_size+test_size+val_size, step=1)
plt.plot(val_data_index,y_pred_val)
prediect_data_index = pd.RangeIndex(start=len(VCB1)-1, stop=len(VCB1)+29, step=1)
plt.plot(prediect_data_index,scaler.inverse_transform(lst_output))
plt.legend(['Train','Test','Test Predictions','Validate','Validate Predictions','Next30days Predictions'])
plt.title("Dự đoán giá cổ phiếu VCB trong 30 ngày kế tiếp theo tỉ lệ 6-3-1")
plt.show()

# 6. Chia tập dữ liệu để train-test-validate theo tỉ lệ 5:3:2

In [11]:
# Split the data into training, testing, and validation sets
train_size = int(0.5 * len(VCB1))
test_size = int(0.3 * len(VCB1))
val_size = len(VCB1) - train_size - test_size

NameError: ignored

In [ ]:
train_data = VCB1[:train_size]
test_data = VCB1[train_size:train_size+test_size]
val_data = VCB1[train_size+test_size:]

<h1>Hàm Create Dataset

In [ ]:
#Chuyển mảng giá trị của dataset thành ma trận
def create_dataset(dataset, time_step=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, X=0,1,2,3-----99   Y=100
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [ ]:
import numpy
#Reshape into X=t,t+1, t+2... t+99 and Y= t+100
time_step =100
X_train, y_train = create_dataset(train_data, time_step)
X_val, yval = create_dataset(val_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)

In [ ]:
#Reshape input to be [samples, time steps, features] which is required for GRU
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)
X_val = X_val.reshape(X_val.shape[0],X_val.shape[1] , 1)

In [ ]:
#Define GRU model
model = Sequential()
model.add(GRU(64, input_shape=(time_step, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
#Model fitting
model.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=100,batch_size=64,verbose=1)

<h1>Dự báo dữ liệu test, val

In [ ]:
train_predict=model.predict(X_train)
y_pred=model.predict(X_test)
y_pred_val=model.predict(X_val)

<h1>Chuẩn hóa dữ liệu

In [12]:
#Chuẩn hóa dữ liệu y_pred, y_pred_val
train_predict=scaler.inverse_transform(train_predict)
y_pred=scaler.inverse_transform(y_pred)
y_pred_val=scaler.inverse_transform(y_pred_val)

NameError: ignored

In [ ]:
ytest = scaler.inverse_transform(ytest.reshape(-1,1))
ytest

In [ ]:
yval = scaler.inverse_transform(yval.reshape(-1,1))
yval

<h1>Đánh giá độ chính xác của thuật toán

In [ ]:
# RMSE
valid_rmse = np.sqrt(np.mean((y_pred_val - yval)**2))
test_rmse = np.sqrt(np.mean((y_pred - ytest)**2))
print('Validation RMSE:', valid_rmse)
print('Testing RMSE:', test_rmse)

In [ ]:
#MAE
valid_mae = mean_absolute_error(yval,y_pred_val)
test_mae = mean_absolute_error(ytest, y_pred)
print('Validation MAE:', valid_mae)
print('Testing MAE:', test_mae)

In [ ]:
#MAPE
valid_mape = np.mean(np.abs((yval - y_pred_val) / yval))*100
test_mape = np.mean(np.abs((ytest - y_pred) / ytest))*100
print('Validation MAPE:', valid_mape)
print('Testing MAPE:', test_mape)

<h1>Quá trình dự đoán giá cổ phiếu trong 30 ngày tiếp theo

In [13]:

lst_output = []
n_steps = 30  # Adjust the sequence length as needed
i = 0

while i < 30:
    if len(temp_input) >= n_steps:
        x_input = np.array(temp_input[-n_steps:])
        x_input = x_input.reshape((1, n_steps, 1))

        # Print for debugging
        print("{} day input {}".format(i, x_input))

        yhat = model.predict(x_input, verbose=0)

        # Print for debugging
        print("{} day output {}".format(i, yhat))

        temp_input.extend(yhat[0].tolist())
        temp_input = temp_input[1:]
        lst_output.extend(yhat.tolist())
        i = i + 1
    else:
        x_input = x_input.reshape((1, len(temp_input), 1))

        # Print for debugging
        print("{} day input {}".format(i, x_input))

        yhat = model.predict(x_input, verbose=0)

        # Print for debugging
        print("{} day output {}".format(i, yhat))

        temp_input.extend(yhat[0].tolist())
        i = i + 1

print(lst_output)


NameError: ignored

In [ ]:
print(scaler.inverse_transform(lst_output))

<h1>Vẽ hình mô tả dự đoán

In [ ]:
#Drawing plot
train_data_index = pd.RangeIndex(start=0, stop=train_size, step=1)
plt.plot(scaler.inverse_transform(train_data))
test_data_index = pd.RangeIndex(start=train_size, stop=train_size+test_size, step=1)
plt.plot(test_data_index,scaler.inverse_transform(test_data))
test_data_index = pd.RangeIndex(start=train_size+101, stop=train_size+test_size, step=1)
plt.plot(test_data_index,(y_pred))
val_data_index = pd.RangeIndex(start=train_size+test_size, stop=train_size+test_size+val_size, step=1)
plt.plot(val_data_index,scaler.inverse_transform(val_data))
val_data_index = pd.RangeIndex(start=train_size+test_size+101, stop=train_size+test_size+val_size, step=1)
plt.plot(val_data_index,y_pred_val)
prediect_data_index = pd.RangeIndex(start=len(VCB1)-1, stop=len(VCB1)+29, step=1)
plt.plot(prediect_data_index,scaler.inverse_transform(lst_output))
plt.legend(['Train','Test','Test Predictions','Validate','Validate Predictions','Next30days Predictions'])
plt.title("Dự đoán giá cổ phiếu VCB trong 30 ngày kế tiếp theo tỉ lệ 5-3-2")
plt.show()